<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Sebastian Raschka가 집필한 <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> 도서를 위한 보조 코드입니다.<br>
<br>코드 저장소: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>

# 4장 연습문제 해답

In [1]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.9.0


# 연습문제 4.1: 피드포워드와 어텐션 모듈의 파라미터 수

- 피드포워드 모듈과 멀티헤드 어텐션 모듈에 포함된 파라미터 수를 계산해 비교해 보라.

In [3]:
from gpt import TransformerBlock

GPT_CONFIG_124M = {
    "vocab_size": 50257,
    "context_length": 1024,
    "emb_dim": 768,
    "n_heads": 12,
    "n_layers": 12,
    "drop_rate": 0.1,
    "qkv_bias": False
}

block = TransformerBlock(GPT_CONFIG_124M)
print(block)

TransformerBlock(
  (att): MultiHeadAttention(
    (W_query): Linear(in_features=768, out_features=768, bias=False)
    (W_key): Linear(in_features=768, out_features=768, bias=False)
    (W_value): Linear(in_features=768, out_features=768, bias=False)
    (out_proj): Linear(in_features=768, out_features=768, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (ff): FeedForward(
    (layers): Sequential(
      (0): Linear(in_features=768, out_features=3072, bias=True)
      (1): GELU()
      (2): Linear(in_features=3072, out_features=768, bias=True)
    )
  )
  (norm1): LayerNorm()
  (norm2): LayerNorm()
  (drop_shortcut): Dropout(p=0.1, inplace=False)
)


In [ ]:
# 피드포워드 모듈의 전체 파라미터 수를 계산하여 출력합니다.
total_params = sum(p.numel() for p in block.ff.parameters())
print(f"피드포워드 모듈의 전체 파라미터 수: {total_params:,}")

피드포워드 모듈의 전체 파라미터 수: 4,722,432


In [5]:
# 어텐션 모듈의 전체 파라미터 수를 계산하여 출력합니다.
total_params = sum(p.numel() for p in block.att.parameters())
print(f"어텐션 모듈의 전체 파라미터 수: {total_params:,}")

어텐션 모듈의 전체 파라미터 수: 2,360,064


- 위 결과는 하나의 트랜스포머 블록에 대한 값입니다.
- 124M GPT 모델의 모든 트랜스포머 블록을 고려하려면 12를 곱하면 됩니다.

**보너스: 수학적 계산 과정**

- 아래는 파라미터 수를 수학적으로 계산하는 과정입니다(`emb_dim=768` 가정).

피드포워드 모듈:
- 1번째 `Linear` 층: 768 입력 × 4×768 출력 + 4×768 편향 = 2,362,368
- 2번째 `Linear` 층: 4×768 입력 × 768 출력 + 768 편향 = 2,360,064
- 합계: 2,362,368 + 2,360,064 = 4,722,432

어텐션 모듈:
- `W_query`: 768 입력 × 768 출력 = 589,824 
- `W_key`: 768 입력 × 768 출력 = 589,824
- `W_value`: 768 입력 × 768 출력 = 589,824 
- `out_proj`: 768 입력 × 768 출력 + 768 편향 = 590,592
- 합계: 3×589,824 + 590,592 = 2,360,064 

# 연습문제 4.2: 더 큰 GPT 모델 초기화하기

- 1억 2,400만 개 파라미터의 GPT 모델(일명 "GPT-2 small")을 초기화했다. 
- 구성 파일만 수정하고 나머지 코드는 그대로 둔 채, GPTModel 클래스를 사용해 다음과 같은 더 큰 모델들을 구현해 보자
- 보너스로 각 모델의 총 파라미터 수도 계산해 보자.

- **GPT2-small**(이미 구현한 124M 구성):
    - "emb_dim" = 768
    - "n_layers" = 12
    - "n_heads" = 12

- **GPT2-medium:**
    - "emb_dim" = 1024
    - "n_layers" = 24
    - "n_heads" = 16

- **GPT2-large:**
    - "emb_dim" = 1280
    - "n_layers" = 36
    - "n_heads" = 20

- **GPT2-XL:**
    - "emb_dim" = 1600
    - "n_layers" = 48
    - "n_heads" = 25

In [6]:
# GPT-2 Small(124M) 모델 기본 구성 정의
GPT_CONFIG_124M = {
    "vocab_size": 50257,      # 어휘 집합 크기
    "context_length": 1024,   # 최대 문맥 길이(토큰 수)
    "emb_dim": 768,           # 임베딩 차원
    "n_heads": 12,            # 어텐션 헤드 개수
    "n_layers": 12,           # 트랜스포머 블록 개수
    "drop_rate": 0.1,         # 드롭아웃 비율
    "qkv_bias": False         # QKV 연산시 bias 사용 여부
}

# 모델 이름에 따라 구성 값을 업데이트하는 함수
def get_config(base_config, model_name="gpt2-small"):
    GPT_CONFIG = base_config.copy()  # 기존 구성 복사

    if model_name == "gpt2-small":
        # Small 구성: 기본값과 동일
        GPT_CONFIG["emb_dim"] = 768
        GPT_CONFIG["n_layers"] = 12
        GPT_CONFIG["n_heads"] = 12

    elif model_name == "gpt2-medium":
        # Medium 구성
        GPT_CONFIG["emb_dim"] = 1024
        GPT_CONFIG["n_layers"] = 24
        GPT_CONFIG["n_heads"] = 16

    elif model_name == "gpt2-large":
        # Large 구성
        GPT_CONFIG["emb_dim"] = 1280
        GPT_CONFIG["n_layers"] = 36
        GPT_CONFIG["n_heads"] = 20

    elif model_name == "gpt2-xl":
        # XL 구성
        GPT_CONFIG["emb_dim"] = 1600
        GPT_CONFIG["n_layers"] = 48
        GPT_CONFIG["n_heads"] = 25

    else:
        raise ValueError(f"Incorrect model name {model_name}")

    return GPT_CONFIG

# 모델의 전체 파라미터 수, weight tying된 파라미터 수, 모델 파일 사이즈를 계산/출력하는 함수
def calculate_size(model): # 본문 코드 기반
    
    total_params = sum(p.numel() for p in model.parameters())
    print(f"전체 파라미터 수: {total_params:,}")

    # weight tying 고려한 파라미터 수 (out_head 파라미터 중복 제외)
    total_params_gpt2 =  total_params - sum(p.numel() for p in model.out_head.parameters())
    print(f"Weight tying을 고려한 학습 가능한 파라미터 수: {total_params_gpt2:,}")
    
    # 파라미터를 float32(1개당 4바이트)로 가정해 전체 바이트 크기 계산
    total_size_bytes = total_params * 4
    
    # 메가바이트 단위로 변환
    total_size_mb = total_size_bytes / (1024 * 1024)
    
    print(f"모델 파일 크기(예상): {total_size_mb:.2f} MB")

In [7]:
from gpt import GPTModel


for model_abbrev in ("small", "medium", "large", "xl"):
    model_name = f"gpt2-{model_abbrev}"
    CONFIG = get_config(GPT_CONFIG_124M, model_name=model_name)
    model = GPTModel(CONFIG)
    print(f"\n\n{model_name}:")
    calculate_size(model)



gpt2-small:
전체 파라미터 수: 163,009,536
Weight tying을 고려한 학습 가능한 파라미터 수: 124,412,160
모델 파일 크기(예상): 621.83 MB


gpt2-medium:
전체 파라미터 수: 406,212,608
Weight tying을 고려한 학습 가능한 파라미터 수: 354,749,440
모델 파일 크기(예상): 1549.58 MB


gpt2-large:
전체 파라미터 수: 838,220,800
Weight tying을 고려한 학습 가능한 파라미터 수: 773,891,840
모델 파일 크기(예상): 3197.56 MB


gpt2-xl:
전체 파라미터 수: 1,637,792,000
Weight tying을 고려한 학습 가능한 파라미터 수: 1,557,380,800
모델 파일 크기(예상): 6247.68 MB


# 연습문제 4.3: 드롭아웃 파라미터 분리해서 사용하기

- 이 장의 앞부분에서 우리는 GPT_CONFIG_124M 딕셔너리에 전역 드롭아웃 비율(drop_rate)을 정의해 GPTModel 아키텍처의 여러 위치에서 드롭아웃 비율을 설정했다. 
- 이제 모델 아키텍처 내의 각 드롭아웃 계층마다 서로 다른 드롭아웃 값을 지정하도록 코드를 변경해 보자.
- 힌트: 드롭아웃 계층은 임베딩 층, 숏컷 연결, 멀티헤드 어텐션 모듈의 세 곳에 사용된다.

In [8]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,             # 어휘 집합 크기
    "context_length": 1024,          # 컨텍스트(입력) 토큰 최대 길이
    "emb_dim": 768,                  # 임베딩 차원 수
    "n_heads": 12,                   # 멀티헤드 어텐션 헤드 개수
    "n_layers": 12,                  # 트랜스포머 블록(레이어) 개수
    "drop_rate_emb": 0.1,            # NEW: 임베딩 층 드롭아웃 비율
    "drop_rate_attn": 0.1,           # NEW: 멀티헤드 어텐션 드롭아웃 비율
    "drop_rate_shortcut": 0.1,       # NEW: 잔차(shortcut) 연결 드롭아웃 비율
    "qkv_bias": False                # 어텐션 QKV 바이어스 사용 여부
}

In [ ]:
import torch.nn as nn
from gpt import MultiHeadAttention, LayerNorm, FeedForward

# 트랜스포머 블록 정의
class TransformerBlock(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # 멀티헤드 어텐션 모듈
        self.att = MultiHeadAttention(
            d_in=cfg["emb_dim"],
            d_out=cfg["emb_dim"],
            context_length=cfg["context_length"],
            num_heads=cfg["n_heads"], 
            dropout=cfg["drop_rate_attn"], # 어텐션 드롭아웃 비율
            qkv_bias=cfg["qkv_bias"])
        # 피드포워드 네트워크
        self.ff = FeedForward(cfg)
        # 첫 번째 레이어 정규화
        self.norm1 = LayerNorm(cfg["emb_dim"])
        # 두 번째 레이어 정규화
        self.norm2 = LayerNorm(cfg["emb_dim"])
        # 잔차(shortcut) 연결용 드롭아웃
        self.drop_shortcut = nn.Dropout(cfg["drop_rate_shortcut"])

    def forward(self, x):
        # --- 어텐션 블록 + 잔차 연결 ---
        shortcut = x                      # 입력을 보존 (잔차)
        x = self.norm1(x)                 # 정규화
        x = self.att(x)                   # 멀티헤드 어텐션 ([batch, tokens, emb_dim])
        x = self.drop_shortcut(x)         # 어텐션 출력에 드롭아웃 적용
        x = x + shortcut                  # 잔차 연결

        # --- 피드포워드 블록 + 잔차 연결 ---
        shortcut = x                      # 입력을 다시 보존 (잔차)
        x = self.norm2(x)                 # 정규화
        x = self.ff(x)                    # 피드포워드 네트워크
        x = self.drop_shortcut(x)         # 드롭아웃 적용
        x = x + shortcut                  # 잔차 연결

        return x

# GPT 모델 전체 구조 정의
class GPTModel(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        # 토큰 임베딩
        self.tok_emb = nn.Embedding(cfg["vocab_size"], cfg["emb_dim"])
        # 위치 임베딩
        self.pos_emb = nn.Embedding(cfg["context_length"], cfg["emb_dim"])
        # 임베딩 드롭아웃
        self.drop_emb = nn.Dropout(cfg["drop_rate_emb"])

        # 여러 트랜스포머 블록을 순차적으로 쌓음
        self.trf_blocks = nn.Sequential(
            *[TransformerBlock(cfg) for _ in range(cfg["n_layers"])])

        # 최종 레이어 정규화
        self.final_norm = LayerNorm(cfg["emb_dim"])
        # 최종 출력(로짓) 레이어
        self.out_head = nn.Linear(cfg["emb_dim"], cfg["vocab_size"], bias=False)

    def forward(self, in_idx):
        batch_size, seq_len = in_idx.shape
        # 입력 인덱스를 임베딩으로 변환
        tok_embeds = self.tok_emb(in_idx)
        # 위치 임베딩 생성 (0 ~ seq_len-1)
        pos_embeds = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        # 토큰 임베딩 + 위치 임베딩 합성
        x = tok_embeds + pos_embeds  # [batch_size, num_tokens, emb_dim]
        x = self.drop_emb(x)         # 임베딩 드롭아웃
        x = self.trf_blocks(x)       # 트랜스포머 블록 통과
        x = self.final_norm(x)       # 마지막 정규화
        logits = self.out_head(x)    # 어휘 크기만큼의 로짓 산출
        return logits

In [ ]:
import torch  # 파이토치 임포트

torch.manual_seed(123)  # 난수 시드 고정 (재현성)
model = GPTModel(GPT_CONFIG_124M)  # GPT 모델 인스턴스 생성